In [76]:
import pandas as pd

In [86]:
df1=pd.read_csv('fakenews.csv')
df2=pd.read_csv('fakenews test.csv')
df=pd.concat([df1,df2],axis=0)
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26000 entries, 0 to 5199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      26000 non-null  int64  
 1   title   25320 non-null  object 
 2   author  23540 non-null  object 
 3   text    25954 non-null  object 
 4   label   20800 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.2+ MB


In [88]:
df.dropna(inplace=True)

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      18285 non-null  int64  
 1   title   18285 non-null  object 
 2   author  18285 non-null  object 
 3   text    18285 non-null  object 
 4   label   18285 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 857.1+ KB


In [90]:
X=df.drop('label',axis=1)


In [91]:
y=df['label']

In [92]:
X.shape

(18285, 4)

In [93]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [94]:
voc_size=5000

In [95]:
messages=X.copy()
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [96]:
messages.reset_index(inplace=True)
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [97]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [99]:
### Dataset Preprocessing
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [100]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [101]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3041  822 3346]
 [   0    0    0 ... 2252 1888  715]
 [   0    0    0 ...  382  534 1667]
 ...
 [   0    0    0 ... 1414 3824 4081]
 [   0    0    0 ...  858 4925 1871]
 [   0    0    0 ...  391 3294 3627]]


In [102]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1453,
          6,  235, 1084, 3086,  359,  606, 3041,  822, 3346])

In [103]:
#model creation
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_4 (LSTM)               (None, 100)               56400     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [104]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [105]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.2,random_state=42)

In [106]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
229/229 [==============================] - 5s 15ms/step - loss: 0.2994 - accuracy: 0.8724 - val_loss: 0.1991 - val_accuracy: 0.9169
Epoch 2/10
229/229 [==============================] - 3s 14ms/step - loss: 0.1332 - accuracy: 0.9498 - val_loss: 0.1975 - val_accuracy: 0.9237
Epoch 3/10
229/229 [==============================] - 3s 14ms/step - loss: 0.0982 - accuracy: 0.9649 - val_loss: 0.2317 - val_accuracy: 0.9223
Epoch 4/10
229/229 [==============================] - 3s 14ms/step - loss: 0.0667 - accuracy: 0.9774 - val_loss: 0.2355 - val_accuracy: 0.9169
Epoch 5/10
229/229 [==============================] - 3s 14ms/step - loss: 0.0478 - accuracy: 0.9839 - val_loss: 0.3382 - val_accuracy: 0.9130
Epoch 6/10
229/229 [==============================] - 3s 14ms/step - loss: 0.0291 - accuracy: 0.9913 - val_loss: 0.3758 - val_accuracy: 0.9180
Epoch 7/10
229/229 [==============================] - 4s 15ms/step - loss: 0.0180 - accuracy: 0.9949 - val_loss: 0.3494 - val_accuracy: 0.9161

In [115]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64,activation='softmax'))
model.add(Dense(32,activation='softmax'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [116]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
229/229 [==============================] - 6s 16ms/step - loss: 0.6751 - accuracy: 0.5656 - val_loss: 0.6417 - val_accuracy: 0.5693
Epoch 2/10
229/229 [==============================] - 3s 14ms/step - loss: 0.5910 - accuracy: 0.7432 - val_loss: 0.5427 - val_accuracy: 0.8909
Epoch 3/10
229/229 [==============================] - 3s 15ms/step - loss: 0.4818 - accuracy: 0.9233 - val_loss: 0.4427 - val_accuracy: 0.9111
Epoch 4/10
229/229 [==============================] - 3s 15ms/step - loss: 0.3792 - accuracy: 0.9445 - val_loss: 0.3751 - val_accuracy: 0.9098
Epoch 5/10
229/229 [==============================] - 3s 14ms/step - loss: 0.3001 - accuracy: 0.9555 - val_loss: 0.3244 - val_accuracy: 0.9158
Epoch 6/10
229/229 [==============================] - 3s 15ms/step - loss: 0.2442 - accuracy: 0.9616 - val_loss: 0.2976 - val_accuracy: 0.9125
Epoch 7/10
229/229 [==============================] - 3s 15ms/step - loss: 0.1950 - accuracy: 0.9701 - val_loss: 0.2784 - val_accuracy: 0.9141

In [117]:
y_pred = model.predict(X_test)
predict=np.argmax(y_pred, axis=1)

115/115 [==============================] - 1s 3ms/step


In [118]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predict)

array([[2082,    0],
       [1575,    0]], dtype=int64)

In [119]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predict)

0.5693191140278917

### *****************

In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [16]:
texts = df['text'].sample(n=2000).tolist()
labels = df['label'].sample(n=2000).tolist()

In [17]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(train_texts)

In [19]:
vocab=len(tokenizer.word_index)+1

In [22]:
train_sequences = tokenizer.texts_to_sequences(train_texts)
train_padded = pad_sequences(train_sequences, padding='post')
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=train_padded.shape[1])

In [23]:
train_labels = np.array(train_labels)

In [24]:
#model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
model=Sequential()
model.add(Embedding(vocab,100,input_length=train_padded.shape[1]))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
 model.fit(train_padded,train_labels,epochs=1,batch_size=32)

34/50 [===================>..........] - ETA: 33:58 - loss: 0.6829 - accuracy: 0.5689

# **********************

In [29]:
df=pd.read_csv('spam.csv',encoding='latin1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [30]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [31]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [36]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
df['v1']=label.fit_transform(df['v1'])

In [37]:
df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [53]:
texts=df['v2'].tolist()
labels=df['v1'].tolist()

In [54]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [55]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
vocab_size = len(tokenizer.word_index) + 1

In [56]:
train_sequences = tokenizer.texts_to_sequences(train_texts)
train_padded = pad_sequences(train_sequences, padding='post')

In [57]:
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=train_padded.shape[1])

In [58]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [59]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=train_padded.shape[1]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [60]:
model.fit(train_padded, train_labels, epochs=10, batch_size=32)

Epoch 1/10
140/140 [==============================] - 20s 129ms/step - loss: 0.4169 - accuracy: 0.8598
Epoch 2/10
140/140 [==============================] - 18s 129ms/step - loss: 0.3961 - accuracy: 0.8661
Epoch 3/10
140/140 [==============================] - 20s 141ms/step - loss: 0.3955 - accuracy: 0.8661
Epoch 4/10
140/140 [==============================] - 20s 145ms/step - loss: 0.3956 - accuracy: 0.8661
Epoch 5/10
140/140 [==============================] - 18s 131ms/step - loss: 0.3949 - accuracy: 0.8661
Epoch 6/10
140/140 [==============================] - 19s 136ms/step - loss: 0.3948 - accuracy: 0.8661
Epoch 7/10
140/140 [==============================] - 19s 134ms/step - loss: 0.3951 - accuracy: 0.8661
Epoch 8/10
140/140 [==============================] - 18s 131ms/step - loss: 0.3950 - accuracy: 0.8661
Epoch 9/10
140/140 [==============================] - 19s 133ms/step - loss: 0.3946 - accuracy: 0.8661
Epoch 10/10
140/140 [==============================] - 18s 132ms/step - l

In [61]:
loss, accuracy = model.evaluate(test_padded, test_labels)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

35/35 [==============================] - 1s 18ms/step - loss: 0.3951 - accuracy: 0.8655
Loss: 0.3951166272163391
Accuracy: 0.865470826625824


In [68]:
l=['this is very fake news and spam']
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded = pad_sequences(new_sequences, padding='post', maxlen=train_padded.shape[1])
model.predict(new_padded)

1/1 [==============================] - 0s 43ms/step


array([[0.12850201],
       [0.12850201]], dtype=float32)

In [65]:
new_texts = ['I ve been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.','This is a fake news article.']
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded = pad_sequences(new_sequences, padding='post', maxlen=train_padded.shape[1])

predictions = model.predict(new_padded)
for text, prediction in zip(new_texts, predictions):
    label = 'Real' if prediction >= 0.5 else 'Fake'
    print(f'Text: {text}')
    print(f'Prediction: {label}')
    print()

1/1 [==============================] - 0s 45ms/step
Text: I ve been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.
Prediction: Fake

Text: This is a fake news article.
Prediction: Fake

